# Standardbibliotheken

Genau wie pandapower stellt auch pandapipes eine Reihe an Standardbibliotheken zur Verfügung, um die Arbeit mit dem Programm zu beschleunigen. Im Unterschied zu pandapower gibt es aber nicht nur eine Bauteil- sondern auch eine Fluidbibliothek, in der nach der Installation von pandapipes bereits einige Fluide abgelegt sind. Natürlich ist es aber auch möglich, neue Fluide zu erzeugen und diese der Bibliothek hinzuzufügen.

Dieses Tutorial demonstriert den grundlegenden Umgang mit den hinterlegten Bibliotheken und zeigt, wie Komponenten und Fluide geladen und gespeichert werden können. Wir beginnen mit der Demonstration der Komponentenbibliothek, über welche pandapipes verschiedene Rohrleitungen und Pumpen zur Verfügung stellt.


## pandapipes-Komponentenbibliothek

Mit Hilfe der Funktion "get_data", welche im Modul std_type_toolbox definiert wird, lassen sich alle aktuell vorhandenen Einträge in der Standardbibliothek einsehen. Neben dem Pfad zur entsprechenden Datei wird in einem zweiten Parameter auch der Komponententyp abgefragt. Im Folgenden werden die Einträge der gespeicherten pipes geplottet.

In [ ]:
from pandapipes.std_types.std_type_toolbox import get_data
print(get_data(r'Pipe.csv', 'pipe'))

Die CSV-Dateien können selbstverständlich erweitert werden, um neue Einträge hinzuzufügen. Wir werden jetzt ein Beispielnetzwerk erzeugen und dabei Komponenten aus der Bibliothek verwenden.

In [ ]:
import pandapipes as pp
net = pp.create_empty_network(fluid="water")
junction1 = pp.create_junction(net, pn_bar=1.0, tfluid_k=293.15, name="Connection to External Grid")
junction2 = pp.create_junction(net, pn_bar=1.0, tfluid_k=293.15, name="Junction")
junction3 = pp.create_junction(net, pn_bar=1.0, tfluid_k=293.15, name="Junction to sink")
medium_pressure_grid = pp.create_ext_grid(net, junction=junction1, p_bar=1, t_k=293.15, name="Grid Connection")
sink = pp.create_sink(net, junction=junction3, mdot_kg_per_s=23.05, name="Sink")

Im obigen Code fehlt noch eine Rohrverbindung zwischen junction1 und junction2. Wir laden dazu den Rohrtyp "300_GGG" aus der Bibliothek. Dazu wird die Funktion "create_pipe" verwendet. Im Gegensatz zur Funktion "create_pipe_from_parameters" aus dem Eingangsbeispiel, kann nun ein Parameter zum Laden eines Typs aus der Bibliothek angegeben werden.

In [ ]:
pipe1 = pp.create_pipe(net, from_junction=junction2, to_junction=junction3, length_km=1, std_type="300_GGG", name="Pipe 1")

Zwischen External Grid und Rohrleitung existiert ebenfalls noch keine Verbindung. Diese wird jetzt durch das Laden einer Pumpenkomponente geschaffen. Das Druckniveau des External Grids wird durch die Pumpe entsprechend der Pumpenkennlinie angehoben.

In [ ]:
P1 = pp.create_pump(net, from_junction=junction1, to_junction=junction2, std_type="P1")

Das erstellte Netzwerk kann jetzt auf gewohnte Art berechnet werden.

In [ ]:
pp.pipeflow(net)
net.res_pump


Die Pumpe erhöht den Druck um 4 bar. Wir können das abschließend nachprüfen, indem wir noch einen Blick auf die Kennlinie der Pumpe werfen.

In [ ]:
print(get_data(r'P1.csv', 'pump'))

Wir erkennen, dass ein Druckanstieg von 4 bar zu einem Volumenstrom von 83 m^3/h gehört. "Zufällig" wird dieser Volumenstrom genau von der definierten sink abgenommen (23.05 kg/s entsprechen 83 m^3/h) und durch die einzige vorhandene Rohrleitung transportiert.

## Die Fluidbibliothek

Bisher wurde das Fluid als Parameter der Funktion "create_empty_network" definiert. Dabei wird intern auf die hinterlegten Materialeigenschaften zurückgegriffen, die unter dem angegebenen Namen gespeichert sind. Einige der hinterlegten Fluidtypen sind

- air
- hydrogen
- lgas
- hgas
- water
- methane

Für die Berechnung des Druckes und der Strömungsgeschwindigkeit muss die Dichte und Viskosität angegeben werden. Bei Gasberechnungen können diese Stoffeigenschaften außerdem temperaturabhängig sein. Für Temperaturberechnungen oder weitere Berechnungsmodi können weitere Stoffeigenschaften, bspw. die Wärmekapazität, benötigt werden.

Einen Überblick über die hinterlegten Eigenschaften und Fluide erhalten Sie unter pandapipes\properties. Im Ordner des entsprechenden Fluids sehen Sie die Stoffeigenschaften, für die aktuell Daten zur Verfügung stehen. In der zugehörigen Textdatei können Stützpunkte für die Temperaturabhängigkeit hinterlegt werden. Während der Berechnung wird zwischen diesen Punkten interpoliert.

Über das aktuell im Netz enthaltende Fluid können Sie auch aus dem Programm heraus einiges in Erfahrung bringen. Dabei hilft die Funktion get_property(), wie im Folgenden demonstriert wird.

In [ ]:
net.fluid.get_property("density", 295)

Der zweite Parameter ist die Temperatur, bei der die Dichte ausgegeben werden soll.

## Neue Fluide definieren

pandapipes bietet die Möglichkeit, neue Fluide zu definieren oder bestehende Fluide zu modifizieren. Die bisher genutzten Fluide besaßen, wie bereits erwähnt, eine Temperaturabhängigkeit. Im Folgenden wird ein neues,inkompressibles Fluid mit konstanten Materialeigenschaften erstellt und anschließend dem Netz hinzugefügt.

In einem ersten Schritt werden dazu die gewünschten Materialeigenschaften erstellt. Diese Eigenschaften werden dann von einem Fluid-Container aufgenommen. Datenstrukturtechnisch handelt es sich sowohl bei den Eigenschaften, als auch bei dem eigentlichen Fluid, um Klassen.

In [ ]:
import pandapipes.properties.fluids as fl
density_const = fl.FluidPropertyConstant(900)
viscosity_const = fl.FluidPropertyConstant(0.002)
heat_capacity_const = fl.FluidPropertyConstant(4000)

Die drei Eigenschaften  Dichte (density), Viskosität (viscosity) und Wärmekapazität (heat_capacity) werden in jedem Fall benötigt. Für jede Eigenschaft wird ein Objekt der Klasse "FluidPropertyConstant" angelegt und der konstante Wert der Eigenschaft dem Konstruktor übergeben. Die Eigenschaften werden alle in den entsprechenden SI-Einheiten definiert.

Obwohl wir hier nur konstante Eigenschaften betrachten, stellt pandapipes auch Klassen zur Verfügung, mit deren Hilfe aus Daten inter- und extrapoliert werden kann. Diese Klassen werden entsprechend auch für die bereits vorhandenen Fluide genutzt, hier aber nicht weiter betrachtet.

Es folgt der zweite Schritt, in dem der Fluid-Container erstellt wird und die bereits definierten Eigenschaften diesem hinzugefügt werden.

In [ ]:
water_const =  fl.Fluid("water_const","liquid")
water_const.add_property(property_name="density",prop=density_const)
water_const.add_property(property_name="viscosity",prop=viscosity_const)
water_const.add_property(property_name="heat_capacity",prop= heat_capacity_const)

Das Fluid ist jetzt für die Berechnung inkompressibler Medien geeignet. Mit dem Befehl ...:

In [ ]:
fl._add_fluid_to_net(net,water_const)

... kann das Fluid dem bereits erstellten Netz hinzugefügt werden. Durch erneutes Plotten der Fluideigenschaften stellen wir fest, dass das alte Fluid ersetzt worden ist.

In [ ]:
net.fluid

## Lessons learned

Nach dem Durcharbeiten dieses Tutorials sollten Sie wissen,

- dass es eine Komponentenbibliothek in pandapipes gibt, aus der Daten für Rohrleitungen und Pumpen entnommen werden können.
- wie Sie Komponenten aus der Bibliothek nutzen können.
- wie Sie neue Einträge der Bibliothek hinzufügen können.
- wie Fluide in pandapipes beschrieben werden.
- wie Sie herausfinden können, welche Fluide pandapipes bereits bereitstellt und wie diese einem Netz hinzugefügt werden können.
- wie Sie eigene Fluide mit konstanten Stoffeigenschaften erzeugen können.

## Aufgaben

1. Wenn Sie mit dem konstanten Fluid erneut eine Berechnung durchführen, so erscheint in der Konsole eine Warnung. Können Sie sich vorstellen, warum diese angezeigt wird?
2. Erzeugen Sie einen Knoten und legen Sie eine Verbindung zwischen diesem und dem zweiten Knoten. Wählen Sie eine andere Rohrleitung aus der Bibliothek aus. Fügen Sie zudem eine Quelle hinzu mit einem Massestrom von 0.1 kg/s. Ändern Sie zudem den Standardtyp der Pumpe. Rechnen Sie erneut.